# Exploratory Data Analysis: Goodreads Reviews

This notebook performs comprehensive EDA on English Goodreads reviews for 6,000 romance novels, analyzing:
- Review count distributions (overall and by quality tier)
- Review length distributions (characters and tokens)
- Ratings distributions
- Lexical patterns by quality tier

**Data Sources:**
- Books: `data/processed/romance_subdataset_6000.csv`
- Reviews: `data/processed/romance_reviews_english_subdataset_6000.csv`
- Coverage: `data/interim/review_coverage_by_book.csv`


In [1]:
# Setup and imports
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from collections import Counter
import re
from typing import Dict, List

# Add project root to path
# Use absolute path to project root (works in both Jupyter and script execution)
project_root = Path('/home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research')

# Add src to path
sys.path.insert(0, str(project_root / "src"))

# Change to project root directory for relative paths to work
import os
os.chdir(project_root)

# Import our modules
from reviews_analysis.data_loading import load_joined_reviews
from reviews_analysis.checks_coverage import generate_coverage_table

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set up plotting style
plt.style.use('default')
sns.set_style("whitegrid")

# Antique color palette (from user preferences)
ANTIQUE_COLORS = [
    '#855C75FF', '#D9AF6BFF', '#AF6458FF', '#736F4CFF', 
    '#526A83FF', '#625377FF', '#68855CFF', '#9C9C5EFF', 
    '#A06177FF', '#8C785DFF', '#467378FF', '#7C7C7CFF'
]

# Create output directory
output_dir = project_root / "reports" / "reviews_eda"
output_dir.mkdir(parents=True, exist_ok=True)

logger.info(f"Output directory: {output_dir}")
logger.info("Setup complete")


INFO:__main__:Output directory: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/reports/reviews_eda


INFO:__main__:Setup complete


In [2]:
# Load data
logger.info("Loading joined reviews and books data...")
joined_df = load_joined_reviews(how="inner")  # Only books with reviews

logger.info(f"Loaded {len(joined_df):,} reviews")
logger.info(f"Unique books: {joined_df['work_id'].nunique():,}")
logger.info(f"Columns: {list(joined_df.columns)}")

# Check data types and basic info
print("\nData shape:", joined_df.shape)
print("\nFirst few rows:")
joined_df.head()

INFO:__main__:Loading joined reviews and books data...


INFO:reviews_analysis.data_loading:Loading books dataset from: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/data/processed/romance_subdataset_6000.csv


INFO:reviews_analysis.data_loading:Loaded 6,000 books


INFO:reviews_analysis.data_loading:Pop tier distribution:
pop_tier
thrash    2000
mid       2000
top       2000
Name: count, dtype: int64


INFO:reviews_analysis.data_loading:Loading reviews dataset from: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/data/processed/romance_reviews_english_subdataset_6000.csv


...

INFO:__main__:Loaded 969,675 reviews


INFO:__main__:Unique books: 5,998


INFO:__main__:Columns: ['review_id', 'review_text', 'rating', 'book_id', 'work_id', 'title', 'author_id', 'author_name', 'publication_year', 'num_pages_median', 'genres_str', 'series_id', 'series_title', 'ratings_count_sum', 'text_reviews_count_sum', 'average_rating_weighted_mean', 'pop_tier']



Data shape: (969675, 17)

First few rows:


,review_id,review_text,rating,book_id,work_id,title,author_id,author_name,publication_year,num_pages_median,genres_str,series_id,series_title,ratings_count_sum,text_reviews_count_sum,average_rating_weighted_mean,pop_tier
0,f73a70f64564d4a8f4cfb2d2e9d5836f,Enjoyable read! I liked that Connie is not a t...,4,7840190,21509416,Bait,3132972,Annie Nicholas,2010,269.0,"fantasy, fiction, mystery, paranormal, romance",406250.0,Angler,2504,230,3.70,thrash
1,768e4a82b89e36055706857e9268b8e9,I really enjoyed this book! Chick lit with bra...,5,2718668,2744265,A Family Affair,363909,Mary Campisi,2006,351.0,"fiction, mystery, romance",542852.0,Truth in Lies,11490,928,3.63,thrash
2,77ecec100fdb475116e9cd75a77dc00d,Eh...a lil harlequin romance anyone?,2,2761356,2787065,The Sweet Gum Tree,1077176,Katherine Allred,2005,304.0,"fiction, romance, young adult",stand_alone,stand_alone,24509,2877,4.30,top
3,63ff74279e46b247cb1754313b160006,I finished reading this days ago and cant get ...,4,15507958,17763198,Me Before You,281810,Jojo Moyes,2012,385.0,"fiction, romance",747511.0,Me Before You,737888,67781,4.27,top
4,d27adfd2eec3cddb66954e87cf9ab197,** spoiler alert ** \n So many people have sai...,5,17183734,21880254,The Edge of Never,5437976,J.A. Redmerski,2012,439.0,"fiction, romance, young adult",471797.0,The Edge of Never,126458,10295,4.26,top


In [3]:
# Compute review lengths (characters and tokens)
logger.info("Computing review lengths...")

# Character count
joined_df['review_length_chars'] = joined_df['review_text'].str.len()

# Token count (simple word count for now)
joined_df['review_length_tokens'] = joined_df['review_text'].str.split().str.len()

# Log basic statistics
print(f"Review length (characters):")
print(f"  Mean: {joined_df['review_length_chars'].mean():.1f}")
print(f"  Median: {joined_df['review_length_chars'].median():.1f}")
print(f"  Min: {joined_df['review_length_chars'].min()}")
print(f"  Max: {joined_df['review_length_chars'].max():,}")

print(f"\nReview length (tokens):")
print(f"  Mean: {joined_df['review_length_tokens'].mean():.1f}")
print(f"  Median: {joined_df['review_length_tokens'].median():.1f}")
print(f"  Min: {joined_df['review_length_tokens'].min()}")
print(f"  Max: {joined_df['review_length_tokens'].max():,}")

INFO:__main__:Computing review lengths...


Review length (characters):
  Mean: 747.8
  Median: 348.0
  Min: 10
  Max: 20,024

Review length (tokens):
  Mean: 137.7
  Median: 65.0
  Min: 1
  Max: 3,789


In [4]:
# 1. Distribution of reviews per book (overall and by pop_tier)
logger.info("Creating review count distribution plots...")

# Get review counts per book
review_counts = joined_df.groupby(['work_id', 'pop_tier']).size().reset_index(name='n_reviews')
review_counts_by_tier = review_counts.groupby('pop_tier')['n_reviews'].apply(list).to_dict()

# Create figure with 2x2 layout
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Review Count Distribution by Quality Tier', fontsize=16, fontweight='bold')

# Overall distribution
ax = axes[0, 0]
ax.hist(review_counts['n_reviews'], bins=50, color=ANTIQUE_COLORS[0], alpha=0.7, edgecolor='black')
ax.set_xlabel('Number of Reviews per Book')
ax.set_ylabel('Frequency')
ax.set_title('Overall Distribution')
ax.set_xlim(0, 1000)  # Focus on most books
ax.grid(True, alpha=0.3)

# By tier - histograms
tier_order = ['thrash', 'mid', 'top']
tier_labels = ['Trash', 'Middle', 'Top']
tier_colors = [ANTIQUE_COLORS[0], ANTIQUE_COLORS[4], ANTIQUE_COLORS[8]]

# Place tiers in remaining 3 subplots: [0,1], [1,0], [1,1]
tier_positions = [(0, 1), (1, 0), (1, 1)]

for idx, (tier, label, color) in enumerate(zip(tier_order, tier_labels, tier_colors)):
    row, col = tier_positions[idx]
    ax = axes[row, col]
    
    if tier in review_counts_by_tier:
        ax.hist(review_counts_by_tier[tier], bins=50, color=color, alpha=0.7, edgecolor='black')
        ax.set_xlabel('Number of Reviews per Book')
        ax.set_ylabel('Frequency')
        ax.set_title(f'{label} Tier')
        ax.set_xlim(0, 1000)
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'review_count_distribution.png', dpi=300, bbox_inches='tight')
plt.close()  # Close to free memory
logger.info(f"Saved: {output_dir / 'review_count_distribution.png'}")

INFO:__main__:Creating review count distribution plots...


INFO:__main__:Saved: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/reports/reviews_eda/review_count_distribution.png


In [5]:
# 2. Review length distributions (characters and tokens) by pop_tier
logger.info("Creating review length distribution plots...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Review Length Distribution by Quality Tier', fontsize=16, fontweight='bold')

# Overall character length
ax = axes[0, 0]
ax.hist(joined_df['review_length_chars'], bins=100, color=ANTIQUE_COLORS[0], alpha=0.7, edgecolor='black')
ax.set_xlabel('Review Length (characters)')
ax.set_ylabel('Frequency')
ax.set_title('Overall Character Length')
ax.set_xlim(0, 5000)  # Focus on most reviews
ax.grid(True, alpha=0.3)

# Character length by tier
tier_positions = [(0, 1), (1, 0), (1, 1)]
for idx, (tier, label, color) in enumerate(zip(tier_order, tier_labels, tier_colors)):
    row, col = tier_positions[idx]
    ax = axes[row, col]
    
    tier_data = joined_df[joined_df['pop_tier'] == tier]['review_length_chars']
    ax.hist(tier_data, bins=100, color=color, alpha=0.7, edgecolor='black')
    ax.set_xlabel('Review Length (characters)')
    ax.set_ylabel('Frequency')
    ax.set_title(f'{label} Tier Character Length')
    ax.set_xlim(0, 5000)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'review_length_chars_distribution.png', dpi=300, bbox_inches='tight')
plt.close()  # Close to free memory
logger.info(f"Saved: {output_dir / 'review_length_chars_distribution.png'}")

INFO:__main__:Creating review length distribution plots...


INFO:__main__:Saved: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/reports/reviews_eda/review_length_chars_distribution.png


In [6]:
# Token length distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Review Token Count Distribution by Quality Tier', fontsize=16, fontweight='bold')

# Overall token length
ax = axes[0, 0]
ax.hist(joined_df['review_length_tokens'], bins=100, color=ANTIQUE_COLORS[0], alpha=0.7, edgecolor='black')
ax.set_xlabel('Review Length (tokens)')
ax.set_ylabel('Frequency')
ax.set_title('Overall Token Count')
ax.set_xlim(0, 1000)  # Focus on most reviews
ax.grid(True, alpha=0.3)

# Token length by tier
tier_positions = [(0, 1), (1, 0), (1, 1)]
for idx, (tier, label, color) in enumerate(zip(tier_order, tier_labels, tier_colors)):
    row, col = tier_positions[idx]
    ax = axes[row, col]
    
    tier_data = joined_df[joined_df['pop_tier'] == tier]['review_length_tokens']
    ax.hist(tier_data, bins=100, color=color, alpha=0.7, edgecolor='black')
    ax.set_xlabel('Review Length (tokens)')
    ax.set_ylabel('Frequency')
    ax.set_title(f'{label} Tier Token Count')
    ax.set_xlim(0, 1000)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'review_length_tokens_distribution.png', dpi=300, bbox_inches='tight')
plt.close()  # Close to free memory
logger.info(f"Saved: {output_dir / 'review_length_tokens_distribution.png'}")

INFO:__main__:Saved: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/reports/reviews_eda/review_length_tokens_distribution.png


In [7]:
# 3. Ratings distribution (if available)
logger.info("Analyzing ratings distribution...")

# Check if ratings are available and valid
ratings_available = 'rating' in joined_df.columns
if ratings_available:
    # Convert rating to numeric, handling empty strings
    joined_df['rating_numeric'] = pd.to_numeric(joined_df['rating'], errors='coerce')
    ratings_valid = joined_df['rating_numeric'].notna().sum()
    
    print(f"Ratings available: {ratings_available}")
    print(f"Valid ratings: {ratings_valid:,} ({ratings_valid/len(joined_df)*100:.1f}%)")
    
    if ratings_valid > 0:
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('Rating Distribution by Quality Tier', fontsize=16, fontweight='bold')
        
        # Overall ratings
        ax = axes[0, 0]
        valid_ratings = joined_df['rating_numeric'].dropna()
        ax.hist(valid_ratings, bins=5, color=ANTIQUE_COLORS[0], alpha=0.7, edgecolor='black', align='left')
        ax.set_xlabel('Rating (stars)')
        ax.set_ylabel('Frequency')
        ax.set_title('Overall Rating Distribution')
        ax.set_xticks([1, 2, 3, 4, 5])
        ax.grid(True, alpha=0.3, axis='y')
        
        # Ratings by tier
        tier_positions = [(0, 1), (1, 0), (1, 1)]
        for idx, (tier, label, color) in enumerate(zip(tier_order, tier_labels, tier_colors)):
            row, col = tier_positions[idx]
            ax = axes[row, col]
            
            tier_ratings = joined_df[joined_df['pop_tier'] == tier]['rating_numeric'].dropna()
            if len(tier_ratings) > 0:
                ax.hist(tier_ratings, bins=5, color=color, alpha=0.7, edgecolor='black', align='left')
                ax.set_xlabel('Rating (stars)')
                ax.set_ylabel('Frequency')
                ax.set_title(f'{label} Tier Ratings')
                ax.set_xticks([1, 2, 3, 4, 5])
                ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(output_dir / 'ratings_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()  # Close to free memory
        
        logger.info(f"Saved: {output_dir / 'ratings_distribution.png'}")
        
        # Print statistics
        print("\nRating statistics by tier:")
        for tier, label in zip(tier_order, tier_labels):
            tier_ratings = joined_df[joined_df['pop_tier'] == tier]['rating_numeric'].dropna()
            if len(tier_ratings) > 0:
                print(f"\n{label}:")
                print(f"  Mean: {tier_ratings.mean():.2f}")
                print(f"  Median: {tier_ratings.median():.2f}")
                print(f"  Count: {len(tier_ratings):,}")
else:
    print("Ratings not available in dataset")


INFO:__main__:Analyzing ratings distribution...


Ratings available: True
Valid ratings: 969,675 (100.0%)


INFO:__main__:Saved: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/reports/reviews_eda/ratings_distribution.png



Rating statistics by tier:

Trash:
  Mean: 3.35
  Median: 4.00
  Count: 121,343



Middle:
  Mean: 3.89
  Median: 4.00
  Count: 480,126



Top:
  Mean: 4.27
  Median: 5.00
  Count: 368,206


In [8]:
# 4. Boxplots comparing distributions by tier
logger.info("Creating comparison boxplots...")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Review Metrics Comparison by Quality Tier', fontsize=16, fontweight='bold')

# Review counts per book
ax = axes[0]
review_counts_pivot = review_counts.pivot(columns='pop_tier', values='n_reviews')
review_counts_pivot.boxplot(ax=ax, positions=[0, 1, 2], widths=0.6, 
                           patch_artist=True,
                           boxprops=dict(facecolor=ANTIQUE_COLORS[0], alpha=0.7),
                           medianprops=dict(color='black', linewidth=2))
ax.set_xticklabels(tier_labels)
ax.set_ylabel('Number of Reviews per Book')
ax.set_title('Review Counts per Book')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 500)  # Focus on most books

# Review length (characters)
ax = axes[1]
data_for_box = [joined_df[joined_df['pop_tier'] == tier]['review_length_chars'].dropna() 
                for tier in tier_order]
bp = ax.boxplot(data_for_box, labels=tier_labels, widths=0.6, patch_artist=True)
for patch, color in zip(bp['boxes'], tier_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax.set_ylabel('Review Length (characters)')
ax.set_title('Review Character Length')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 3000)

# Review length (tokens)
ax = axes[2]
data_for_box = [joined_df[joined_df['pop_tier'] == tier]['review_length_tokens'].dropna() 
                for tier in tier_order]
bp = ax.boxplot(data_for_box, labels=tier_labels, widths=0.6, patch_artist=True)
for patch, color in zip(bp['boxes'], tier_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax.set_ylabel('Review Length (tokens)')
ax.set_title('Review Token Count')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 600)

plt.tight_layout()
plt.savefig(output_dir / 'review_metrics_boxplots.png', dpi=300, bbox_inches='tight')
plt.close()  # Close to free memory
logger.info(f"Saved: {output_dir / 'review_metrics_boxplots.png'}")


INFO:__main__:Creating comparison boxplots...


/tmp/ipykernel_147381/3085279190.py:24: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot(data_for_box, labels=tier_labels, widths=0.6, patch_artist=True)


/tmp/ipykernel_147381/3085279190.py:37: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot(data_for_box, labels=tier_labels, widths=0.6, patch_artist=True)


INFO:__main__:Saved: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/reports/reviews_eda/review_metrics_boxplots.png


In [9]:
# 5. Basic lexical analysis: frequent words by tier
logger.info("Performing lexical analysis...")

import string
from collections import Counter

def clean_text(text):
    """Basic text cleaning for word frequency analysis."""
    if pd.isna(text):
        return ""
    # Convert to lowercase and remove punctuation
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Common stopwords (basic list)
stopwords = {
    'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 
    'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'were', 'been',
    'be', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could',
    'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those',
    'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them',
    'my', 'your', 'his', 'her', 'its', 'our', 'their', 'book', 'books', 'read',
    'reading', 'review', 'reviews', 'story', 'character', 'characters', 'plot'
}

def get_top_words(text_series, n=20):
    """Get top N words from a series of texts."""
    all_words = []
    for text in text_series:
        cleaned = clean_text(text)
        words = cleaned.split()
        all_words.extend([w for w in words if w not in stopwords and len(w) > 2])
    
    word_counts = Counter(all_words)
    return word_counts.most_common(n)

# Get top words by tier
print("Top 20 words by quality tier:\n")
top_words_by_tier = {}
for tier, label in zip(tier_order, tier_labels):
    tier_reviews = joined_df[joined_df['pop_tier'] == tier]['review_text']
    top_words = get_top_words(tier_reviews, n=20)
    top_words_by_tier[tier] = top_words
    
    print(f"{label} Tier:")
    for word, count in top_words:
        print(f"  {word}: {count:,}")
    print()


INFO:__main__:Performing lexical analysis...


Top 20 words by quality tier:



Trash Tier:
  not: 98,154
  just: 74,181
  like: 73,910
  one: 68,864
  all: 66,686
  about: 65,541
  really: 64,768
  love: 64,074
  what: 61,848
  more: 59,375
  out: 52,389
  when: 51,968
  there: 51,667
  who: 44,777
  how: 41,217
  because: 39,039
  very: 38,205
  get: 37,270
  good: 37,047
  some: 36,628



Middle Tier:
  not: 344,097
  love: 312,676
  just: 300,194
  one: 300,144
  like: 280,678
  all: 267,650
  really: 247,735
  about: 238,200
  more: 234,854
  what: 227,652
  when: 199,323
  out: 193,218
  there: 186,509
  loved: 184,124
  how: 172,400
  who: 161,971
  because: 156,182
  much: 150,936
  get: 146,580
  series: 145,332



Top Tier:
  love: 300,890
  not: 261,739
  one: 253,467
  just: 239,834
  all: 235,587
  like: 198,433
  more: 195,689
  series: 191,338
  what: 189,751
  about: 183,809
  really: 168,281
  loved: 166,615
  when: 160,010
  out: 154,294
  how: 150,407
  there: 148,294
  much: 129,021
  get: 127,532
  because: 122,695
  who: 117,831



In [10]:
# Visualize top words by tier
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Top 15 Words by Quality Tier', fontsize=16, fontweight='bold')

for idx, (tier, label, color) in enumerate(zip(tier_order, tier_labels, tier_colors)):
    ax = axes[idx]
    
    if tier in top_words_by_tier:
        words, counts = zip(*top_words_by_tier[tier][:15])
        words = list(words)[::-1]  # Reverse for horizontal bar
        counts = list(counts)[::-1]
        
        ax.barh(range(len(words)), counts, color=color, alpha=0.7)
        ax.set_yticks(range(len(words)))
        ax.set_yticklabels(words)
        ax.set_xlabel('Frequency')
        ax.set_title(f'{label} Tier')
        ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(output_dir / 'top_words_by_tier.png', dpi=300, bbox_inches='tight')
plt.close()  # Close to free memory
logger.info(f"Saved: {output_dir / 'top_words_by_tier.png'}")


INFO:__main__:Saved: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/reports/reviews_eda/top_words_by_tier.png


In [11]:
# Summary statistics table
logger.info("Generating summary statistics...")

summary_stats = []

for tier, label in zip(tier_order, tier_labels):
    tier_data = joined_df[joined_df['pop_tier'] == tier]
    
    stats = {
        'Tier': label,
        'Total Reviews': len(tier_data),
        'Unique Books': tier_data['work_id'].nunique(),
        'Mean Reviews per Book': tier_data.groupby('work_id').size().mean(),
        'Median Reviews per Book': tier_data.groupby('work_id').size().median(),
        'Mean Review Length (chars)': tier_data['review_length_chars'].mean(),
        'Median Review Length (chars)': tier_data['review_length_chars'].median(),
        'Mean Review Length (tokens)': tier_data['review_length_tokens'].mean(),
        'Median Review Length (tokens)': tier_data['review_length_tokens'].median(),
    }
    
    if 'rating_numeric' in tier_data.columns:
        valid_ratings = tier_data['rating_numeric'].dropna()
        if len(valid_ratings) > 0:
            stats['Mean Rating'] = valid_ratings.mean()
            stats['Median Rating'] = valid_ratings.median()
            stats['Reviews with Ratings'] = len(valid_ratings)
    
    summary_stats.append(stats)

summary_df = pd.DataFrame(summary_stats)
print("\nSummary Statistics by Quality Tier:")
print(summary_df.to_string(index=False))

# Save summary
summary_df.to_csv(output_dir / 'summary_statistics.csv', index=False)
logger.info(f"Saved summary statistics to: {output_dir / 'summary_statistics.csv'}")


INFO:__main__:Generating summary statistics...


INFO:__main__:Saved summary statistics to: /home/polina/Documents/goodreads_romance_research_cursor/romance_novel_nlp_research/reports/reviews_eda/summary_statistics.csv



Summary Statistics by Quality Tier:
  Tier  Total Reviews  Unique Books  Mean Reviews per Book  Median Reviews per Book  Mean Review Length (chars)  Median Review Length (chars)  Mean Review Length (tokens)  Median Review Length (tokens)  Mean Rating  Median Rating  Reviews with Ratings
 Trash         121343          1999              60.701851                     29.0                  757.478042                         375.0                   138.793824                           70.0     3.349546            4.0                121343
Middle         480126          2000             240.063000                    134.0                  731.877563                         338.0                   134.635696                           63.0     3.885505            4.0                480126
   Top         368206          1999             184.195098                     79.0                  765.396740                         352.0                   141.422492                           66.0     4

## Summary

This EDA has analyzed:
1. ✅ Review count distributions (overall and by tier)
2. ✅ Review length distributions (characters and tokens)
3. ✅ Ratings distributions (if available)
4. ✅ Lexical patterns (top words by tier)
5. ✅ Summary statistics

**Key Findings:**
- Coverage: 5,998/6,000 books (99.97%) have reviews
- Total reviews: 969,675
- Clear differences in review patterns across quality tiers
- MID tier has highest average reviews per book
- Review lengths vary by tier

All figures saved to `reports/reviews_eda/`
